In [1]:
%%script false --no-raise-error
import json
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud.bigquery import magics

credentials_json = userdata.get('BIGQUERY_CREDENTIALS')
credentials = service_account.Credentials.from_service_account_info(json.loads(credentials_json))
magics.context.credentials = credentials

Couldn't find program: 'false'


In [8]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
%load_ext bigquery_magics

data_set = "testing_set"
project_name = "emerald-entity-468916-f9"

job_config = bigquery.QueryJobConfig(default_dataset = f"{project_name}.{data_set}", job_timeout_ms = 3600000)
client = bigquery.Client(project = project_name, default_query_job_config = job_config, credentials = globals().get('credentials', None))
magics.context.default_query_job_config = job_config
magics.context.project = project_name

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


In [3]:
%%bigquery

call phase3_prepare_chunks();

Query is running:   0%|          |

""


In [4]:
%%bigquery

call phase4_fragments_summarization();
call phase4_summarize_fragments_summaries();

Query is running:   0%|          |

""


In [5]:
%%bigquery

call phase5_prepare_character_ids_from_fragments();
call phase5_prepare_character_ids_initial_embeddings();

Query is running:   0%|          |

""


In [6]:
%%bigquery

call phase5_merge_characters_duplicates();
call phase5_merge_characters_duplicates();
delete from identifiers where importance <= 1;
delete from tmp_characters_id_embeddings where id not in (select id from identifiers);

call phase5_merge_characters_duplicates();
call phase5_merge_characters_duplicates();
delete from identifiers where importance <= 3;
delete from tmp_characters_id_embeddings where id not in (select id from identifiers);

call phase5_merge_characters_duplicates();
call phase5_merge_characters_duplicates();
delete from identifiers where importance <= 5;
delete from tmp_characters_id_embeddings where id not in (select id from identifiers);

BEGIN
  DECLARE merged INT64 DEFAULT 0;
  REPEAT
    call phase5_merge_characters_duplicates_with_return_param(merged);
  UNTIL merged = 0
  END REPEAT;
END;

call phase5_rebuild_indentifier_jsons();

Executing query with job ID: 42143ac0-a48e-4291-b956-ca332f2f6254
Query executing: 3544.70s


ERROR:
 499 GET https://bigquery.googleapis.com/bigquery/v2/projects/emerald-entity-468916-f9/queries/42143ac0-a48e-4291-b956-ca332f2f6254?maxResults=0&location=US&prettyPrint=false: Job execution was cancelled: User requested cancellation

Location: US
Job ID: 42143ac0-a48e-4291-b956-ca332f2f6254



In [13]:
%%bigquery

CALL phase6_gather_characters_full_data_from_chunks();

CALL phase6_group_the_same_characters_data_for_merging();
CALL phase6_merge_data();

CALL phase6_copy_final_data();

Executing query with job ID: 5a9adbb3-b238-4cd6-8ae3-e0e6697f1391
Query executing: 720.64s

KeyboardInterrupt: 

In [ ]:
%%bigquery

UPDATE characters SET social_class_cluster_id = NULL, wealth_cluster_id = NULL, values_cluster_id = NULL WHERE 1 = 1;
DELETE FROM clusters WHERE 1 = 1;
DELETE FROM character_cluster_details WHERE 1 = 1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --params {"clusters": 10, "trait_type": "values", "trait_desc": 'values: "Core principles, moral compass, priorities (can include both positive and negative values)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.values as traits from characters c WHERE c.values is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET values_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Executing query with job ID: 2a8d7599-63a6-4c1a-ae1a-651f7da583a2
Query executing: 73.00s

KeyboardInterrupt: 

In [ ]:
%%bigquery --params {"clusters": 7, "trait_type": "wealth", "trait_desc": 'wealth: "Economic position, assets, property, financial struggles or abundance with information how wealth/income is obtained (inheritance, labor, trade, crime, patronage, etc.)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.wealth as traits from characters c WHERE c.wealth is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET wealth_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --params {"clusters": 7, "trait_type": "social_class", "trait_desc": 'social_class: "Economic and social standing (e.g., nobility, working class, merchant class)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.social_class as traits from characters c WHERE c.social_class is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET social_class_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Query is running:   0%|          |

""
